In [6]:
import fastjet as fj
import time
import pyhepmc

In [7]:
def makeJets(data, R):
    jet_def = fj.JetDefinition(fj.cambridge_aachen_algorithm, R)
    jet_cluster = fj.ClusterSequence(data, jet_def)

    return jet_cluster

In [8]:
def read_hepmc_file(FILE):
    total = []
    with pyhepmc.open(f"./data/{FILE}", "r") as f:
        
        for i, event in enumerate(f):
            particles = []
            for j, particle in enumerate(event.particles):
                if particle.status == 1: 
                # Access particle properties such as momentum, energy, etc.
                # print(particle.momentum.px)
                    px, py, pz = particle.momentum.px, particle.momentum.py, particle.momentum.pz
                    energy = particle.momentum.e

                    particles.append(fj.PseudoJet(px, py, pz, energy))
            total.append(particles)

    return total

In [9]:
fileName = ["events-ee-Z.hepmc3", "events-ee-H.hepmc3", \
                "events-pp-0.5TeV-5GeV.hepmc3", "events-pp-1TeV-5GeV.hepmc3",\
                "events-pp-2TeV-5GeV.hepmc3", "events-pp-2TeV.hepmc3",\
                "events-pp-5TeV-10GeV.hepmc3", "events-pp-8TeV-20GeV.hepmc3",\
                "events-pp-13TeV-20GeV.hepmc", "events-pp-20TeV-20GeV.hepmc3", "events-pp-20TeV-50GeV.hepmc3",\
                "events-pp-30TeV-50GeV.hepmc3"]

In [10]:
trials = 16

with open("./results/time_python_plain.csv", "w") as FILE:
    FILE.write("FileName,mean_particles,n_samples,time_per_event\n")
    for eventFILE in fileName:
        allEvents = read_hepmc_file(eventFILE)

        avgTime = 0.0
        for j in range(trials):
            
            count = 0
            start = time.time()
            for i in range(100):
                count += len(allEvents[i])
                # cluster = makeJets(allEvents[i], 0.4, fj.N2Plain)
                cluster = makeJets(allEvents[i], 0.4)
            
            end = time.time() - start
            avgTime += end

            # count += len(particle_data)
        avgTime /= trials

        print(f"{eventFILE}\t{count/100}\t{trials}\t{avgTime*10**4}\n")
        FILE.write(f"{eventFILE},{count/100},{trials},{avgTime*10**4}\n")


#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
events-ee-Z.hepmc3	43.05	16	22.349655628204346

events-ee-H.hepmc3	64.9

In [ ]:
trials = 64
with open("./results/time-python-substructure.csv", "w") as FILE:
    FILE.write("FileName,mean_particles,n_samples,filter_time,trimming_time,massdrop_time\n")
    total_filter = total_trim = total_softdrop = total_massdrop = 0.0
    total_count = 0

    for eventFILE in fileName:
        allEvents = read_hepmc_file(eventFILE)

        avgTime = 0.0
        avg_filter_time = 0.0
        avg_trimming_time = 0.0
        avg_massdrop_time = 0.0

        n_events = 100
        
        for j in range(trials):

            count = 0
            
            for i in range(n_events):
                cluster = makeJets(allEvents[i], 1.0)
                jets = cluster.inclusive_jets(2.0)
                count += len(jets)
                # mu = 0.67
                # ycut = 0.09
                mass_drop_tagger = fj.MassDropTagger(0.67, 0.09)
                start = time.time()
                final_jets = [mass_drop_tagger(jet) for jet in jets]
                end = time.time()
                avg_massdrop_time += (end - start) * 10**6

                filtering = fj.Filter(fj.JetDefinition(fj.cambridge_algorithm, 0.3), fj.SelectorNHardest(3))
                start = time.time()
                final_jets = [filtering(jet) for jet in jets]
                end = time.time()
                avg_filter_time += (end - start) * 10**6

                trimming = fj.Filter(fj.JetDefinition(fj.cambridge_algorithm, 0.3), fj.SelectorPtFractionMin(0.3))
                start = time.time()
                final_jets = [trimming(jet) for jet in jets]
                end = time.time()
                avg_trimming_time += (end - start) * 10**6

                # JH_tagger = fj.JHTopTagger(0.1, 0.19, 0.7, 80.4)
                # start = time.time()
                # final_jets = [JH_tagger(jet) for jet in jets]                
                # end = time.time() - start

        total_filter += avg_filter_time
        total_massdrop += avg_massdrop_time
        total_trim += avg_trimming_time
        total_count += count

        avg_massdrop_time /= (trials * count)
        avg_filter_time /= (trials * count)
        avg_trimming_time /= (trials * count)

        print(f"{eventFILE}\t{count/100}\t{trials}\t{avg_filter_time}\t{avg_trimming_time}\t{avg_massdrop_time}\n")
        FILE.write(f"{eventFILE},{count/100},{trials},{avg_filter_time},{avg_trimming_time},{avg_massdrop_time}\n")

events-ee-Z.hepmc3	2.67	64	2.1554780810066823	1.9004282433441961	0.9821511609723952

events-ee-H.hepmc3	3.04	64	2.2490704922299636	1.9737298747426586	1.0078503309111846

events-pp-0.5TeV-5GeV.hepmc3	6.68	64	2.1607241409267495	1.9594636595177792	0.8524724555586627

events-pp-1TeV-5GeV.hepmc3	9.53	64	2.2639679758646305	2.0529203124711795	0.8401839142706312

events-pp-2TeV-5GeV.hepmc3	11.24	64	2.2849286790420154	2.0686265253511613	0.8358073245163914

events-pp-2TeV.hepmc3	12.35	64	2.461165069085866	2.2407998198922345	0.8735066724692279

events-pp-5TeV-10GeV.hepmc3	17.08	64	2.380155149053355	2.152650710849628	0.826722181094614

events-pp-8TeV-20GeV.hepmc3	18.89	64	2.498796151516108	2.2685420519463784	0.8420437937252354

events-pp-13TeV-20GeV.hepmc	23.33	64	2.526861375562633	2.3105741824605053	0.8368841502168789

events-pp-20TeV-20GeV.hepmc3	27.6	64	2.5452991974526555	2.329842443915381	0.8276137320891671

events-pp-20TeV-50GeV.hepmc3	26.32	64	2.5958451230112907	2.36061398834443	0.8531311090

In [15]:
print(f"{total_count}")
print(f"Filter: {total_filter/(total_count*trials)}")
print(f"Trim: {total_trim/(total_count*trials)}")
print(f"Massdrop: {total_massdrop/(total_count*trials)}")

18843
Filter: 2.4875172737177054
Trim: 2.264816560911245
Massdrop: 0.8477472100599249
